# Use examples of toolbox


#### toolbox.dao.connector.DBconnector

In [1]:
from toolbox.dao.connector import DBConnector,parse_db_access

In [2]:
db_access = parse_db_access("./examples/db_connector_testing/db.ini","BASE")

DBC = DBConnector(db_access,scen_id=1) 
# for a lot of values required to assign for variables in sql queries  
# the following argument is recommended; vars_in_sql can also be a return of configuration file using configparser package

# vars_in_sql = {
#     "scen_id":1
# }
# DBC = DBConnector(db_access,**vars_in_sql)

DBC.set_queries_dir("./examples/db_connector_testing/mssql_queries")
df = DBC.pull_predefined_query("wafer_eq.scen_query")
df

,scen_id,scen_name,type,status,instance_id,locked
0,1,Central Baseline – DMTM,Cap Standard MOR,mizer ...,1,False


In [2]:
db_access = parse_db_access("./examples/db_connector_testing/db.ini","XEUS")
DBC = DBConnector(db_access)
DBC.set_queries_dir("./examples/db_connector_testing/oracle_queries")
#DBC.set_cache_dir(DBC.queries_dir,cache_mode=1)
df = DBC.pull_predefined_query("test")
print("pull by predefined SQL query:")
display(df)

s = """
select * from f_lot_run_card 
FETCH first 5 ROWS only
"""
df = DBC.pull_SQL(s)
print("pull by sql string")
display(df)


In [2]:
from toolbox.dao.connector import parse_db_access, BlobConnector
import pandas as pd
db_access = parse_db_access("./examples/db_connector_testing/db.ini","optisuitestorage")
BC = BlobConnector(db_access)

In [3]:
import azure

class BlobVirtualFolder:
    def __init__(self,path:str):
        self.path=path
        self.elements=[
            x.name for x in BC.blob_container_client.walk_blobs(path)
        ]

        self._folders = [
            x.name for x in BC.blob_container_client.walk_blobs(path)
            if isinstance(x,azure.storage.blob._list_blobs_helper.BlobPrefix)
        ]
        
        if path is not None:
            self.parent_path = self._get_parent_folder()
        else:
            self.parent_path = "root"

    def _get_parent_folder(self):
        if len(self.path.split("/"))<=2:
            return None
        else:
            temp = self.path.split("/")
            return "/".join(temp[:-2])+"/"
    
    @property
    def folders(self):
        return self._folders

    @folders.setter
    def folders(self,f_list:list):
        self._folders = f_list

BVF={}
def walk_folder(path):
    if path not in BVF.keys():
        BVF[path] = BlobVirtualFolder(path)

    if len(BVF[path].folders)>0:
        walk_folder(BVF[path].folders.pop())
    elif BVF[path].parent_path != "root":
            return walk_folder(BVF[path].parent_path)
    else:
            return None


walk_folder(None)



In [27]:
import os
from IPython.core.display import HTML

df = HTML(pd.DataFrame(map(lambda x:['<img src="file.png" width="60" >',x] if os.path.split("/")[-1] =="" else ["folder.png",x] ,BVF['testwafercost/'].elements),columns=["type","blob name"]).to_html(escape=False))

In [28]:
type(df)

IPython.core.display.HTML

#### download the blob to a specified location (dir_path is default to ".")

In [2]:
import os
scen_name='implanttestavail'
display(BC.get_blob_list(scen_name))
#blob_name='implanttestavail/solverData/2022-11-16 09:01:54 PM.zip'
latest_blob = max([os.path.splitext(x)[0] for x in BC.get_blob_list(scen_name) if x.endswith(".zip")]) + '.zip'
latest_blob

['implanttestavail',
 'implanttestavail/solverData',
 'implanttestavail/solverData/2022-11-16 09:01:54 PM.zip',
 'implanttestavail/solverData/2022-11-16 09:02:36 PM.zip',
 'implanttestavail/solverData/2022-11-16 09:03:12 PM.zip']

'implanttestavail/solverData/2022-11-16 09:03:12 PM.zip'

In [3]:
_ = BC.blob_dump(latest_blob, dir_path = "./examples")

#### return Python object for developing purposes 

In [3]:
# return in-memory zip
BC.blob_dump(latest_blob, to_memory=True)


<zipfile.ZipFile file=<_io.BytesIO object at 0x7f321dca36f0> mode='r'>

In [4]:
# return toolbox.dao.feed.attrDict
BC.blob_dump(latest_blob, to_attrDict=True)

{
  "avaliable keys": [
    "Solve_details",
    "gb_per_die",
    "ACL Change Log",
    "Finance Wafer Output",
    "Available Tools Forecasted"
  ]
}

#### upload to Blob storage from dataframe

In [4]:
#upload to Blob storage from a dataframe instance
df = pd.read_pickle("./examples/db_connector_testing/oracle_queries/test.pkl")
BC.upload_csv_from_df(df,'implanttestavail/wafer_cost/','example1.zip',archive=True)
#upload a file to Blob storage from local machine
#BC.file_upload(file_path="example2.csv")